### Host linkages:
We have 38,956 viral sequences (vOTUs), but what is the point of all these if we don't link to hosts?
Would like to link the viruses to hosts, and get AMG predictions if possible

After host linkages: Would like to do co-occurence calculations for viruses and their hosts (sourmash?) to see if the linkages make sense. 

How to:
- download own iphop db bc want to add own MAGs
- run gtdbtk against MAGs, need the taxonomy folder output for adding to iphop
- add MAGs to iphop db
- do linkages

for AMGs:
- Rerun VS2 with dramV setting
- Run DRAM-V


Tried a bunch of CRISPR detection software, so far the most user friendly is Minced. 
Minced outputs a fasta file with all spacer sequences, that we can use to link MAGs to vOTUs. 
We will only be able to link MAGs that have spacers predicted. 

For MAGs that have CRISPR-spacer arrays predicted, maybe run prodigal/prokka or annotate the protein file, so that we can get an idea of the Cas genes. 

In [ ]:
# Using Minced for CRISPR-spacers detection on the MAGs. 
# https://github.com/ctSkennerton/minced
mamba activate minced

# srun
srun --account=ctbrowngrp -p med2 -J ccf -t 5:00:00 -c 12 --mem=50gb --pty bash

mamba activate branchwater
snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 12 --rerun-incomplete -k -s ccf.smk


In [ ]:
# Try running prokka on the MAGs with a CRISPR array
# And use diff settings for bacteria and archeae
# Update for a prokka with a tsv output

srun --account=ctbrowngrp -p med2 -J prokka -t 00:20:00 -c 16 --mem=50gb --pty bash

# This works, now we can look for Cas genes in these MAGs.
mamba activate prokka
prokka genomes/SRR8960980_MAG14.fasta \
--outdir prokka/SRR8960980_MAG14.new --prefix SRR8960980_MAG14 \
--kingdom bacteria, --norrna --notrna --cpus 16 --addgenes



ath_rumvir_24_9902

In [ ]:
# Programs I couldn't get to work.. 
# cctyper??
# https://github.com/Russel88/CRISPRCasTyper
# dependencies missing? Or not working together..

mamba activate cctyper
cctyper genome.fa my_output

# idk about this one
# Crisprcasfinder
# MINCED output not in fa format, so hassle
# level 4 crispr arrays have highest confidence score. 
#https://www.nature.com/articles/s41564-023-01347-5#Sec12 (Use ccf, only use level 4 conf, std settings)

try crisprcasfinder
in /programs/CRISPRCasFinder
perl CRISPRCasFinder.pl

# Build a snake file for all MAGs

perl /home/amhorst/programs/CRISPRCasFinder/CRISPRCasFinder.pl \
-i {infile} -o {outdir} -keep -cas -cpuM {threads}

In [ ]:
# get iphop db (in home/amhorst/databases)
mamba activate iphop_env

iphop download --db_dir ./iphop/
# db in: /home/amhorst/databases/iphop/Aug_2023_pub_rw

# remove the tar file (after at least checking if works)
rm ./iphop/iPHoP_db_Aug23_rw.tar.gz

In [ ]:
# run gtdbtk
# srun
srun --account=ctbrowngrp -p med2 -J gtdbtk -t 24:00:00 -c 100 --mem=120gb --pty bash

# activate and run (pplacer needs mem so get scratch dir)
mamba activate gtdbtk
# gtdbtk classify_wf --cpus 36 --scratch_dir /home/amhorst/pplacer_scratch \
# --genome_dir ../drep.999/dereplicated_genomes/ \
# --extension fasta --skip_ani_screen --out_dir ./

# Gotta redo gtdbtk with classify de novo or some shit, classify_wf does not work
# bacteria
gtdbtk de_novo_wf --genome_dir ../drep.999/dereplicated_genomes/ --bacteria \
--outgroup_taxon p__Patescibacteria --out_dir pig_gut_MAGs_gtdbtk \
--cpus 100 --force --extension fasta
# archeae
gtdbtk de_novo_wf --genome_dir ../drep.999/dereplicated_genomes/ --archaea \
--outgroup_taxon p__Altiarchaeota --out_dir pig_gut_MAGs_gtdbtk \
--cpus 100 --force --extension fasta

# or can i decorate the trees instead?
gtdbtk decorate --input_tree gtdbtk_classify/classify/gtdbtk.bac120.classify.tree.8.tree \
--output_tree gtdbtk_decorate/gtdbtk.bac120.decorated.tree.8.tree \
--gtdbtk_classification_file gtdbtk_classify/gtdbtk.bac120.summary.tsv

# made structure same as in example folder
# with an infer folder where the tree-taxonomy files are. Don't know if that works. 
# the problem is that the bacterial tree has 8 output trees. why??
# Now just using the backbone tree --> renamed and removed the word backbone. 
# with denovo MAGs, may meed to use the denovo command

In [ ]:
# add new MAGs:
# iPHoP v1.3.3
# works if you have one tree for bacteria and one tree for archeae. 
# Use the backbone tree from gtdbtk
# 
# https://bitbucket.org/srouxjgi/iphop/issues/7/no-new-spacers-in-added-custom-mags
srun --account=ctbrowngrp -p med2 -J iphop_add -t 14:00:00 -c 50 --mem=50gb --pty bash

mamba activate iphop_env

iphop add_to_db \
-f /group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/MAGs/drep.999/dereplicated_genomes/ \
-g /group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/MAGs/gtdbtk_decorate/ \
-o /home/amhorst/databases/iphop/pig_gut_MAGs_July2024/ \
-d /home/amhorst/databases/iphop/Aug_2023_pub_rw \
-t 50 

In [ ]:
# Run iphop with new MAGs added
srun --account=ctbrowngrp -p med2 -J iphop -t 100:00:00 -c 60 --mem=70gb --pty bash

mamba activate iphop_env
iphop predict -f hq_virseqs.95.cluster.fa \
-o ./iphop -d /home/amhorst/databases/iphop/pig_gut_MAGs_July2024/ -t 60

In [ ]:
# create env
mamba create -n vibrant
mamba activate vibrant

# isntall
mamba install vibrant

In [ ]:
# Link the db to resources folder (resources/VIBRANT_db)
ln -s /group/jbemersogrp/databases/vibrant/databases/ .

# activate and run in srun
srun --account=ctbrowngrp -p med2 -J vibrant -t 14:00:00 -c 20 --mem=50gb --pty bash

mamba activate vibrant
VIBRANT_run.py \
-i hq_virseqs.95.cluster.fa -folder VIBRANT \
-t 20 -d /group/jbemersogrp/databases/vibrant/databases